In [6]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('../static/data/movie_test.csv')
df.fillna('', inplace=True)
print(df.shape)
df.head(1)

(1964, 17)


,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated,morphs
0,20210397,가가린 (Gagarine),SF,장편 | 예술영화 | SF | 97분 57초 | 12세이상관람가 | 프랑스,2022-12-22,2020년,파니 리에타르|제레미 트루일,피네건 올드필드 | 드니 라방 | 알세니 바틸리 | 리나 쿠드리,https://www.kobis.or.kr/common/mast/movie/2022...,"가가린 옥상에서 하늘을 보며 우주비행사를 꿈꾸던 10대 소년, 유리.\n사랑하는 사...",장편,예술영화,SF,프랑스,97분 57초,12세이상관람가,가가린 옥상 에서 하늘 보며 우주비행사 꿈꾸던 10 대 소년 유리 사랑 하는 사람 ...


In [8]:
df['total'] = df.morphs + (' ' + df.m_genre) * 3 + (' ' + df.movie_director) * 2 + (' ' + df.star_actor) * 2

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.total)
total_tv.shape

(1964, 27791)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')
total_cv = cvect.fit_transform(df.total)
total_cv.shape

(1964, 27791)

In [11]:
indices = pd.Series(df.index, index=df.code)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)
cosine_sim.shape

(1964, 1964)

In [13]:
cosine_sim_cv = cosine_similarity(total_cv)
cosine_sim_cv.shape

(1964, 1964)

In [14]:
def get_recommendation(code, cos_sim):
    index = indices[code]
    sim_scores = pd.Series(cos_sim[index])
    movie_indices = sim_scores.sort_values(ascending=False).head(11).tail(10).index
    return df.code.iloc[movie_indices], df.title.iloc[movie_indices]

In [15]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)
cosine_sim_total.shape

(1964, 1964)

In [17]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total_cv = linear_kernel(total_cv, total_cv)
cosine_sim_total_cv.shape

(1964, 1964)

In [19]:
df.code

0       20210397
1       20209771
2       20192817
3       20163603
4       20231496
          ...   
1959    20218541
1960    20070115
1961    20112018
1962    20159646
1963    20100076
Name: code, Length: 1964, dtype: object

In [26]:
code = "20112018"

In [27]:
get_recommendation(code, cosine_sim_cv)

(984     20223268
 615     20200627
 312     20147527
 42      20080513
 866     20228854
 1423    20198291
 74      20231009
 1081    20205001
 27      20208616
 536     20227066
 Name: code, dtype: object,
 984                        더 컨테이젼: 바이러스의 습격 (The Leakers)
 615                             나의 끝, 당신의 시작 (Relativity)
 312     그날 본 꽃의 이름을 우리는 아직 모른다 (Anohana: The Flower We...
 42                     갓파 쿠와 여름방학을 (Summer Days With Coo)
 866                              단청낭자 (Dan Qing Xiao Jie)
 1423                                      로켓맨 (Rocketman)
 74                거울 속 외딴 성 (Lonely Castle in the Mirror)
 1081                           도망친 여자 (The Woman Who Ran)
 27                                                    간이역
 536                                           깐부(코로나방학소동)
 Name: title, dtype: object)

In [28]:
get_recommendation(code, cosine_sim)

(984     20223268
 258     20217573
 866     20228854
 521     20212281
 74      20231009
 504     20211305
 746     20194967
 603     20175587
 1515    20217675
 503     20218763
 Name: code, dtype: object,
 984               더 컨테이젼: 바이러스의 습격 (The Leakers)
 258          굴뚝마을의 푸펠 (Poupelle of Chimney Town)
 866                     단청낭자 (Dan Qing Xiao Jie)
 521                                         긴 하루
 74       거울 속 외딴 성 (Lonely Castle in the Mirror)
 504                             기문환생 (My Lord 1)
 746                        네레우스: 바다의 괴물 (Nereus)
 603                        나비잠 (Butterfly Sleep)
 1515    링: 더 라스트 챕터 (The Perilous Internet Ring)
 503                          기문마계 (The Dark War)
 Name: title, dtype: object)

In [29]:
# for code in df.code.values:
sim_scores = pd.Series(cosine_sim[indices[code]])
score = sim_scores.sort_values(ascending=False).head(10).tail(9)
score

984     0.115193
258     0.114856
866     0.102914
521     0.101608
74      0.096968
504     0.096827
746     0.093273
603     0.092870
1515    0.088050
dtype: float64

In [30]:
sim_scores = pd.Series(cosine_sim_cv[indices[code]])
score = sim_scores.sort_values(ascending=False).head(10).tail(9)
score

984     0.283509
615     0.268526
312     0.259926
42      0.251488
866     0.248669
1423    0.243087
74      0.240014
1081    0.235070
27      0.233760
dtype: float64